# Librerie
caricamento librerie

In [63]:
import pandas as pd

### Carica DataSet
games_march2025_cleaned

In [64]:
dataset = pd.read_csv("DataSet/games_march2025_cleaned.csv")

#dataset

# Data Understanding

In [78]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 89618 entries, 0 to 89617
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   release_date      89618 non-null  object 
 1   required_age      89618 non-null  int64  
 2   price             89618 non-null  float64
 3   windows           89618 non-null  bool   
 4   mac               89618 non-null  bool   
 5   linux             89618 non-null  bool   
 6   achievements      89618 non-null  int64  
 7   developers        89618 non-null  object 
 8   publishers        89618 non-null  object 
 9   categories        89618 non-null  object 
 10  genres            89618 non-null  object 
 11  estimated_owners  89618 non-null  object 
dtypes: bool(3), float64(1), int64(2), object(6)
memory usage: 6.4+ MB


score_rank troppi null 
metacritic_url non significativo



In [79]:
dataset.describe()
#statistiche del dataset

,required_age,price,achievements
count,89618.000000,89618.000000,89618.000000
mean,0.183624,7.309623,20.552333
std,1.725594,13.331073,163.562418
min,-1.000000,0.000000,0.000000
25%,0.000000,0.990000,0.000000
50%,0.000000,4.990000,5.000000
75%,0.000000,9.990000,20.000000
max,21.000000,999.980000,9821.000000


In [80]:
#dataset.describe(include=['O'])
dataset['user_score'].value_counts()
dataset['metacritic_score'].value_counts()
dataset['score_rank'].value_counts()
dataset['positive'].value_counts()

KeyError: 'user_score'

* Eliminiamo colonna 'appid', id interno al dataset di steam, non utile alla predizione
* Eliminiamo colonna della descrizione e raccomandazioni, sono testuali e non utili alla predizione.
    * 'detailed_description' , 'about_the_game', 'short_description', 'reviews'
* Eliminiamo url dell'immagine del gioco
    * 'header_image'
* Eliminiamo colonne testuali per supporto clienti e url vari
    * 'website', 'support_url', 'support_email', 'metacritic_url', 'notes'
* Eliminiamo colonne per trailer video e foto del gioco
    * 'movies' , 'screenshots'
* Elimino i punteggi che hanno troppi valori null
    * 'user_score', 'score_rank'
* Elimino le liste delle lingue supportate e dei pacchetti che non danno utilità
    * 'supported_languages', 'full_audio_languages', 'packages'
* Elimino numero di dlc e sconti che non danno informazioni o con molti valori nulli
    * 'discount', 'dlc_count', 'tags'
* Elimino dall’input tutte le variabili post-lancio
    * 'recommendations', 'average_playtime_forever', 'average_playtime_2weeks', 'median_playtime_2weeks', 'median_playtime_forever'
    * 'positive', 'negative'
    * 'peak_ccu', 'pct_pos_total', 'num_reviews_total', 'pct_pos_recent', 'num_reviews_recent
        * hanno varibile nulla => 800

In [ ]:
# drop elimina colonna, per eliminare più colonne contemporaneamente [ , ]
dataset.drop('appid', axis=1, inplace=True)
dataset.drop('name', axis=1, inplace=True)
dataset.drop(['detailed_description','about_the_game', 'short_description', 'reviews', 'header_image'], axis=1, inplace=True)
dataset.drop(['website', 'support_url', 'support_email', 'metacritic_url', 'notes'], axis=1, inplace=True)
dataset.drop(['screenshots','movies'], axis=1, inplace=True)
dataset.drop(['user_score', 'score_rank', 'metacritic_score'], axis=1, inplace=True)
dataset.drop(['supported_languages', 'full_audio_languages', 'packages', 'tags'], axis=1, inplace=True)
dataset.drop(['discount', 'dlc_count'], axis=1, inplace=True)
dataset.drop(['recommendations', 'average_playtime_forever', 'average_playtime_2weeks'], axis=1, inplace=True)
dataset.drop(['median_playtime_2weeks', 'median_playtime_forever'], axis=1, inplace=True)
dataset.drop(['positive', 'negative'], axis=1, inplace=True)
dataset.drop(['peak_ccu', 'pct_pos_total', 'num_reviews_total', 'pct_pos_recent', 'num_reviews_recent'], axis=1, inplace=True)

### Salvo su nuovo dataset da poter usare comodamente

In [66]:
# salvo dataset modificato in uno nuovo per poter ripartire da lì
dataset.to_csv('DataSet/games1.csv', index=False)


# ds
##### Salvo le modifiche in un nuovo dataset per poterlo utilizzare
* Inizializzare da QUI

In [67]:
#NUOVO DATASET CON MODIFICHE

ds = pd.read_csv("DataSet/games1.csv")

### Taglio dataset sugli ultimi 10 anni:
* taglio tutti i giochi usciti prima del 2015
    * dataset arriva fino al 2025

In [68]:
ds['release_date'] = pd.to_datetime(ds['release_date'], format='%Y-%m-%d', errors='coerce')

cutoff_date = pd.Timestamp('2015-01-01')

# Filtriamo solo le righe dal 2015 in poi
ds = ds[ds['release_date'] >= cutoff_date]

print(f"Righe rimanenti: {len(ds)}")


Righe rimanenti: 86847


## Variabile Target num giocatori che hanno acquistato il gioco
* per decretare successo del gioco

In [69]:
ds['estimated_owners'] = ds['estimated_owners'].apply(lambda x: x.split('-')[0])
ds['estimated_owners'].value_counts()

estimated_owners
0            67288
20000         9144
50000         4130
100000        2586
200000        2022
500000         833
1000000        441
2000000        263
5000000         78
10000000        34
20000000        18
50000000        10
Name: count, dtype: int64

### Raggruppamento
* Faccio raggruppamento
    * raggruppo per 4 valori quindi 4 intervalli

In [70]:
# dopo il taglio era diventato stringhe
ds['estimated_owners'] = pd.to_numeric(ds['estimated_owners'], errors='coerce')

intervalli = [0, 5000, 50000, 500000, float('inf')]
etichette = ['insuccesso', 'basso_successo', 'medio_successo', 'alto_successo']

ds['success_class'] = pd.cut(ds['estimated_owners'], bins=intervalli, labels=etichette, include_lowest=True)

# elimino colonna 'estimated_owners' non serve più - sostituita da 'success_class'
ds.drop('estimated_owners', axis=1, inplace=True)

print(ds['success_class'].value_counts())

success_class
insuccesso        67288
basso_successo    13274
medio_successo     5441
alto_successo       844
Name: count, dtype: int64


In [71]:
ds.to_csv('DataSet/games1.csv', index=False)

### Eseguo Undersampling
* Dataset troppo sbilanciato meglio eseguire undersampling e portare tutte le classi di successo allo stesso valore della più piccola.

In [ ]:
from imblearn.under_sampling import RandomUnderSampler

# X = tutte le colonne tranne la target
X = ds.drop(columns=['success_class'])

# y = la target
y = ds['success_class']

undersampler = RandomUnderSampler( sampling_strategy='auto', random_state=42)

# Eseguo Undersampling
X_resampled, y_resampled = undersampler.fit_resample(X, y)

# Ricombino in un nuovo DataFrame
ds = pd.concat([X_resampled, y_resampled], axis=1)

print("Distribuzione originale:")
print(y.value_counts())

print("\nDistribuzione dopo undersampling:")
print(y_resampled.value_counts())


# Lettura da games1.csv

In [72]:
ds = pd.read_csv("DataSet/games1.csv")
#print(ds)

## Analisi Frequenze
* Analizzo frequenze
    * categorie generi publishers developers

In [73]:
#print(dataset['genres'])
print(ds['genres'])

0        ['Action', 'Adventure', 'Massively Multiplayer...
1                                  ['Action', 'Adventure']
2                                               ['Action']
3        ['Action', 'Adventure', 'Indie', 'Massively Mu...
4                  ['Action', 'Adventure', 'Free To Play']
                               ...                        
86842                            ['Indie', 'Early Access']
86843         ['Action', 'Adventure', 'Indie', 'Strategy']
86844                            ['Indie', 'Early Access']
86845       ['Adventure', 'RPG', 'Simulation', 'Strategy']
86846                                  ['Action', 'Indie']
Name: genres, Length: 86847, dtype: object


## Analisi Generi
* pulizia 'genres'

In [ ]:
import re
import string
import matplotlib.pyplot as plt
from collections import Counter

def rimuovi_spazi(match):
    return match.group(0).replace(" ", "")

def clean_text(text):
    # Rimuovi gli spazi extra
    text = ' '.join(text.split())
    # Rimuovi la punteggiatura
    text = text.translate(str.maketrans('', '', string.punctuation))
    # Rimuovi i numeri
    text = re.sub(r'\d+', '', text)
    # Rimuovi i caratteri speciali e simboli
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    return text

def tagDistribution(all_genres_list, top_n=35):
    # Conta le ripetizioni di ogni tag
    genres_counts = Counter(all_genres_list)

    # Estrai i top N tag più comuni e i loro conteggi
    top_genres = genres_counts.most_common(top_n)
    other_count = sum(genres_counts.values()) - sum(count for genre, count in top_genres)

    # Seleziona i tag e i conteggi per il grafico
    genres = [genre for genre, count in top_genres] + ['Altro']
    counts = [count for tag, count in top_genres] + [other_count]

    # Crea il grafico a barre della distribuzione dei Generi
    plt.figure(figsize=(20, 15))
    plt.bar(genres, counts, color='skyblue')
    plt.xlabel('Generi')
    plt.ylabel('Numero di occorrenze')
    plt.title('Distribuzione dei Generi')
    plt.xticks(rotation=45)  # Ruota le etichette sull'asse x per una migliore leggibilità
    plt.show()

def frequenze_osservate_genres(ds):
    # Calcola le statistiche descrittive del genres
    genres_stats = ds['genres'].describe()

    # Stampa le statistiche descrittive del genres
    print("Statistiche descrittive del genres:")
    print(genres_stats)

print(ds['genres'])

# Applica la rimozione degli spazi a ogni riga della colonna 'genres'
ds['genres'] = ds['genres'].apply(lambda x: re.sub(r"'([^']*)'", rimuovi_spazi, x))
# Pulisce il dataset dei genres
ds['genres'] = ds['genres'].apply(clean_text)
# Estrai tutti i genres in un'unica stringa
all_genres_string = ds['genres'].str.cat(sep=' ')
# Dividi la stringa in una lista di singoli genres
all_genres_list = all_genres_string.split()

# Rimuove i duplicati convertendo la lista in un set e poi di nuovo in una lista
unique_genres = list(set(all_genres_list))
unique_genres.sort()

# Calcola la distribuzione dei Generi
print("\nDISTRIBUZIONE VARIABILI")
tagDistribution(all_genres_list)

print("\ngenres")
frequenze_osservate_genres(ds)

print(ds['genres'])


# Feature engineering
## feature selection basata sulla frequenza globale dei generi
* Calcola frequenze per ogni genere 
* Ordina generi per ogni riga in base alla frequenza
    * sceglie i primi 2 generi più frequenti 
    * crea colonna 1 con genere più frequente e colonna 2 con il secondo genere più frequente

In [74]:
import ast
import numpy as np
import pandas as pd
from collections import Counter
import itertools

# Parser robusto: assicura che ogni cella diventi una LISTA di generi #preso da stack
def parse_genres_cell(x):
    if isinstance(x, list):
        return [str(s).strip() for s in x if str(s).strip() != ""]
    if pd.isna(x):
        return []
    if isinstance(x, str):
        s = x.strip()
        # stringa che rappresenta una lista Python
        if s.startswith('[') and s.endswith(']'):
            try:
                v = ast.literal_eval(s)
                if isinstance(v, list):
                    return [str(t).strip() for t in v if str(t).strip() != ""]
            except Exception:
                pass
        # fallback: separo per virgola oppure tengo il token singolo
        if ',' in s:
            parts = [p.strip().strip("'").strip('"') for p in s.split(',')]
            return [p for p in parts if p]
        return [s.strip("'").strip('"')]
    # qualunque altro tipo sconosciuto
    return []

# Crea colonna genres ripulita
ds['genres_list'] = ds['genres'].apply(parse_genres_cell)

# Frequenze globali dei generi
all_genres = list(itertools.chain.from_iterable(ds['genres_list']))
freq = Counter(all_genres)

# Per ogni riga: prendi i 2 generi con frequenza globale più alta
# tie-break: mantieni l'ordine originale della riga
def top_two_from_row(genres):
    if not genres:
        return pd.Series([np.nan, np.nan], index=['genre_top1','genre_top2'])
    # rimuove eventuali duplicati mantenendo l'ordine della riga
    uniq = list(dict.fromkeys(genres))
    pos = {g: i for i, g in enumerate(uniq)}
    # ordina per (frequenza globale, ordine nella riga)
    ordered = sorted(uniq, key=lambda g: (-freq.get(g, 0), pos[g]))
    if len(ordered) == 1:
        return pd.Series([ordered[0], np.nan], index=['genre_top1','genre_top2'])
    return pd.Series(ordered[:2], index=['genre_top1','genre_top2'])

ds[['genre_top1','genre_top2']] = ds['genres_list'].apply(top_two_from_row)

print(ds[['genres','genres_list','genre_top1','genre_top2']].head())
print(ds)


                                              genres  \
0  ['Action', 'Adventure', 'Massively Multiplayer...   
1                            ['Action', 'Adventure']   
2                                         ['Action']   
3  ['Action', 'Adventure', 'Indie', 'Massively Mu...   
4            ['Action', 'Adventure', 'Free To Play']   

                                         genres_list genre_top1 genre_top2  
0  [Action, Adventure, Massively Multiplayer, Fre...     Action  Adventure  
1                                [Action, Adventure]     Action  Adventure  
2                                           [Action]     Action        NaN  
3  [Action, Adventure, Indie, Massively Multiplay...      Indie     Action  
4                  [Action, Adventure, Free To Play]     Action  Adventure  
      release_date  required_age  price  windows    mac  linux  achievements  \
0       2017-12-21             0   0.00     True  False  False            37   
1       2015-04-13            17   0.00  

In [76]:
ds.drop(['genres', 'genres_list'], axis=1, inplace=True)

In [77]:
ds.to_csv('DataSet/games1.csv', index=False)

## feature selection di platform
* unificare windows/mac/linux facendone la somma e salvando in un'unica variabile 'platform'

In [10]:
ds['platform_count'] = ds[['windows', 'mac', 'linux']].sum(axis=1).astype(int)

ds.drop(['windows', 'mac', 'linux'], axis=1, inplace=True)

ds['platform_count'].value_counts()

platform_count
1    67353
2    10838
3     8656
Name: count, dtype: int64

In [6]:
import pandas as pd
ds = pd.read_csv("DataSet/games1.csv")

In [11]:
ds.to_csv("DataSet/games1.csv")

# Encoding delle variabili
* Ci sono colonne che contengono liste di variabili.
    * Eseguo Encoding su 'genres' e 'categories'

### Encoding di genres e categories

In [ ]:
import ast
import numpy as np
from sklearn.preprocessing import MultiLabelBinarizer

# Per 'genres'
# Converto le stringhe in liste
ds['genres'] = ds['genres'].apply(ast.literal_eval)

# mlb classe contiene ordine dei generi
mlb = MultiLabelBinarizer()
genres_encoded = mlb.fit_transform(ds['genres'])

# Salvo in una colonna come array
ds['genres_vector'] = list(genres_encoded)

# Per 'categories'
ds['categories'] = ds['categories'].apply(ast.literal_eval)

mlb_categories = MultiLabelBinarizer()
categories_encoded = mlb_categories.fit_transform(ds['categories'])

ds['categories_vector'] = list(categories_encoded)

# esempio
print(ds['genres_vector'].iloc[0])
print(ds['categories_vector'].iloc[0])


### Encoding di developers e publishers

In [ ]:
import ast
import numpy as np
from sklearn.preprocessing import MultiLabelBinarizer

ds['developers'] = ds['developers'].apply(ast.literal_eval)
mlb_developers = MultiLabelBinarizer()
developers_encoded = mlb_developers.fit_transform(ds['developers'])
ds['developers_vector'] = list(developers_encoded)


ds['publishers'] = ds['publishers'].apply(ast.literal_eval)
mlb_publishers = MultiLabelBinarizer()
publishers_encoded = mlb_publishers.fit_transform(ds['publishers'])
ds['publishers_vector'] = list(publishers_encoded)

# esempio
print(mlb_developers.classes_)
print(ds['developers_vector'].iloc[0])
print(ds['publishers_vector'].iloc[0])

In [ ]:
ds.drop(['genres', 'categories'], axis=1, inplace=True)
ds.drop(['developers', 'publishers'], axis=1, inplace=True)

In [ ]:
ds.to_csv('DataSet/games1.csv', index=False)

Estrarre feature dalla data
non serve tutta la data, ma il periodo dell'anno in cui è stato pubblicato. 

In [ ]:
from sklearn.preprocessing import MultiLabelBinarizer

ds['release_date'] = pd.to_datetime(ds['release_date'])
ds['month'] = ds['release_date'].dt.month

# Mappo i mesi in 4 trimestri
def trimestre(month):
    if month in [12, 1, 2]:
        return "winter"
    elif month in [3, 4, 5]:
        return "spring"
    elif month in [6, 7, 8]:
        return "summer"
    else:
        return "autumn"

ds['trimester'] = ds['month'].apply(trimestre)

mlb_trimester = MultiLabelBinarizer()
trimester_encoded = mlb_trimester.fit_transform(ds['trimester'].apply(lambda x: [x])) # prendo le singole 4 possibili opzioni
ds['trimester_vector'] = list(trimester_encoded)

print(mlb_trimester.classes_)
print(ds['trimester_vector'].iloc[0])

In [ ]:
ds.drop('trimester', axis=1, inplace=True)
ds.drop('release_date', axis=1, inplace=True)

In [ ]:
from sklearn.preprocessing import MultiLabelBinarizer

mlb_success_class = MultiLabelBinarizer()
success_class_encoded = mlb_success_class.fit_transform(ds['success_class'].apply(lambda x: [x])) # prendo le singole 4 possibili opzioni
ds['success_class_vector'] = list(success_class_encoded)

print(mlb_success_class.classes_)
print(ds['success_class'].iloc[0])
print(ds['success_class_vector'].iloc[0])

In [ ]:
ds.drop('success_class', axis=1, inplace=True)

In [ ]:
from sklearn.feature_extraction import FeatureHasher

def vector_to_dict(vec):
    return {f"f{i}": int(x) for i, x in enumerate(vec) if x == 1}

fh = FeatureHasher(n_features=1, input_type='dict')

hashed_genres = fh.transform(ds['genres_vector'].apply(vector_to_dict))
ds['genres_hash'] = hashed_genres.toarray().ravel()

hashed_publishers = fh.transform(ds['publishers_vector'].apply(vector_to_dict))
ds['publishers_vector'] = hashed_publishers.toarray().ravel()

hashed_categories = fh.transform(ds['categories_vector'].apply(vector_to_dict))
ds['categories_hash'] = hashed_categories.toarray().ravel()

hashed_developers = fh.transform(ds['developers_vector'].apply(vector_to_dict))
ds['developers_hash'] = hashed_developers.toarray().ravel()

hashed_success_class = fh.transform(ds['success_class_vector'].apply(vector_to_dict))
ds['success_class_hash'] = hashed_success_class.toarray().ravel()

hashed_trimester = fh.transform(ds['trimester_vector'].apply(vector_to_dict))
ds['trimester_hash'] = hashed_trimester.toarray().ravel()

print(ds)

# Matrice di Correlazione
* per asciugare ciò che non è correlato

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score

# Separiamo i nostri dati in dati di training e dati di test
# Specifichiamo la proporzione fra i due con test_size; in questo caso abbiamo impostato il training set al 90% e il test set al 10%
X = ds.drop(columns=['success_class_vector'])  # tutte le colonne tranne il target
y = ds['success_class_vector']  
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.1, random_state=42)

# Utilizziamo il training set per addestrare il modello
model = DecisionTreeClassifier()
model.fit(X_train, y_train)